In [ ]:
!pip install pydicom
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##連結google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir('/content/drive/My Drive/DeepX_G/image_preprocessing') #切換目錄
os.listdir() #確認目錄內容

['BAD', 'GOOD', '.DS_Store', 'test', 'image_preprocessing.ipynb']

#transfer.py

In [5]:
import pydicom
import cv2
import os

# set the path to the folder containing the DICOM images
folder_path_bad = "BAD/bad(.dcm)"
folder_path_good = "GOOD/good(.dcm)"

for file_name in os.listdir(folder_path_bad):
    if file_name.endswith(".dcm"):
        try:
            ds = pydicom.dcmread(os.path.join(folder_path_bad, file_name))
        except pydicom.errors.InvalidDicomError:
            continue

        data = ds.pixel_array
        image = cv2.normalize(data, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

        # loop over a range of angles in degrees
        for angle in range(0, 1):
            # calculate the rotation matrix
            rotation_matrix = cv2.getRotationMatrix2D((image.shape[1]/2, image.shape[0]/2), angle, 1.0)

            # apply the rotation matrix to the image
            rotated = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))

            # adjust the brightness of the rotated image
            bright_image = cv2.convertScaleAbs(rotated, alpha=1.2, beta=0)

            # create a new folder for each PNG image
            folder_name = os.path.join(os.getcwd(), "BAD", "bad(.png)")
            os.makedirs(folder_name, exist_ok=True)

            # save the enhanced image as a PNG file inside the folder
            file_prefix = os.path.splitext(file_name)[0]
            cv2.imwrite(os.path.join(folder_name, f"bad_{file_prefix}.png"), bright_image)
            #cv2.imwrite(os.path.join(folder_name, f"bad_{file_prefix}_{angle}.png"), bright_image)

for file_name in os.listdir(folder_path_good):
    if file_name.endswith(".dcm"):
        try:
            ds = pydicom.dcmread(os.path.join(folder_path_good, file_name))
        except pydicom.errors.InvalidDicomError:
            continue

        data = ds.pixel_array
        image = cv2.normalize(data, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

        # loop over a range of angles in degrees
        for angle in range(0, 1):
            # calculate the rotation matrix
            rotation_matrix = cv2.getRotationMatrix2D((image.shape[1]/2, image.shape[0]/2), angle, 1.0)

            # apply the rotation matrix to the image
            rotated = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))

            # adjust the brightness of the rotated image
            bright_image = cv2.convertScaleAbs(rotated, alpha=1.2, beta=0)

            # create a new folder for each PNG image
            folder_name = os.path.join(os.getcwd(), "GOOD", "good(.png)")
            os.makedirs(folder_name, exist_ok=True)

            # save the enhanced image as a PNG file inside the folder
            file_prefix = os.path.splitext(file_name)[0]
            cv2.imwrite(os.path.join(folder_name, f"good_{file_prefix}.png"), bright_image)
            #cv2.imwrite(os.path.join(folder_name, f"good_{file_prefix}_{angle}.png"), bright_image)


#crop.py

In [8]:
import cv2
import os

bad_img_folder = 'BAD/bad(.png)'
bad_txt_folder = 'BAD/bad_label'

good_img_folder = 'GOOD/good(.png)'
good_txt_folder = 'GOOD/good_label'

# create new folders to save the cropped images
bad_save_folder = 'BAD/bad_crop_result'
good_save_folder = 'GOOD/good_crop_result'
os.makedirs(bad_save_folder, exist_ok=True)
os.makedirs(good_save_folder, exist_ok=True)

# loop through each image file in the bad image folder
for img_file in os.listdir(bad_img_folder):
    # check if the file is an image file
    if not img_file.endswith('.png'):
        continue

    # read the corresponding text file
    txt_file = os.path.splitext(img_file)[0] + '.txt'
    txt_path = os.path.join(bad_txt_folder, txt_file)
    if not os.path.exists(txt_path):
        continue

    # read the image
    img_path = os.path.join(bad_img_folder, img_file)
    img = cv2.imread(img_path)

    # read the bounding box and class label from the text file
    with open(txt_path, 'r') as f:
        line = f.readline()
        class_id, x_center, y_center, width, height = [float(x) for x in line.split()]

    # calculate the coordinates of the bounding box
    x_min = int((x_center - (width / 2)) * img.shape[1])
    y_min = int((y_center - (height / 2)) * img.shape[0])
    x_max = int((x_center + (width / 2)) * img.shape[1])
    y_max = int((y_center + (height / 2)) * img.shape[0])

    # crop the image and save it to the save folder
    cropped_img = img[y_min:y_max, x_min:x_max]
    save_path = os.path.join(bad_save_folder, img_file)
    cv2.imwrite(save_path, cropped_img)

# loop through each image file in the good image folder
for img_file in os.listdir(good_img_folder):
    # check if the file is an image file
    if not img_file.endswith('.png'):
        continue

    # read the corresponding text file
    txt_file = os.path.splitext(img_file)[0] + '.txt'
    txt_path = os.path.join(good_txt_folder, txt_file)
    if not os.path.exists(txt_path):
        continue

    # read the image
    img_path = os.path.join(good_img_folder, img_file)
    img = cv2.imread(img_path)

    # read the bounding box and class label from the text file
    with open(txt_path, 'r') as f:
        line = f.readline()
        class_id, x_center, y_center, width, height = [float(x) for x in line.split()]

    # calculate the coordinates of the bounding box
    x_min = int((x_center - (width / 2)) * img.shape[1])
    y_min = int((y_center - (height / 2)) * img.shape[0])
    x_max = int((x_center + (width / 2)) * img.shape[1])
    y_max = int((y_center + (height / 2)) * img.shape[0])

    # crop the image and save it to the save folder
    cropped_img = img[y_min:y_max, x_min:x_max]
    save_path = os.path.join(good_save_folder, img_file)
    cv2.imwrite(save_path, cropped_img)